In [1]:
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt

import numpy as np
from scipy.integrate import odeint



In [2]:
k0 = 4.1e5
kd = 1.86e5
kq = 1.31e10

def get_constants(T=-30):
    Tk = T + 273.15
    R = 8.314462618153
    
    k0Ea, k0A = 5.126043*1e3, np.exp(1.50257E+01)
    kqEa, kqA = 8.133688*1e3, np.exp(2.6645E+01)
    kdEa, kdA = 25.63632243*1e3, np.exp(22.52314)
    
    k0 = k0A * np.exp(-k0Ea/(R * Tk))
    kq = kqA * np.exp(-kqEa/(R * Tk))
    kd = kdA * np.exp(-kdEa/(R * Tk))
    
    return k0, kq, kd

k0, kq, kd = get_constants(T=50)

print(f'k0={k0:.1E}, kq={kq:.1E}, kd={kd:.1E}')


k0=5.0E+05, kq=1.8E+10, kd=4.3E+05


In [3]:
def lower_lim(conc, k0, kq, kd):
    k1 = kq * conc
    return (np.log(2) - np.log((k0+k1+kd) / (k0+k1)))/ (k0+k1-kd)


In [4]:
def upper_lim(conc, k0, kq, kd):
#     k1 = kq * conc
    x = np.linspace(0, 6e-6, 5000)
    
    maxima = np.zeros(conc.shape[0])
    
    for i, k1 in enumerate(kq*conc):
        df =  np.exp(-x*(k0 + k1))*(np.exp(-x*(k0 + k1)) + np.exp(-kd*x)*(k0*x + k1*x - kd*x - 1))
        m = np.argmax(df)
        
        maxima[i] = x[m]
        
        
    return maxima

In [5]:
def general(conc, kex, k_ex=0, kdel=1e6, kTT=1e10, k0=4.0e5, kd=1.86e5, kq=1.34e10, A0=0,
            t_max=4e-6, intensity=False, curves=False, norm_curves=False, t_points=1000):
    
    def Tall(A):
        result = (1 - np.power(10, -A)) / (A * np.log(10))
        result[np.isnan(result)] = 1
        return result
    
    def solve(conc, t, *args):
        Xan, Ex, DR, DF = conc
        k1_var = args[0]

        dcXan_dt = -(k0 + k1_var) * Xan
        dcEx_dt = + k1_var * Xan  - (kex + kd) * Ex + k_ex * DR
        dcDR_dt = + kex * Ex - kd * DR - k_ex * DR
        # DF simulation
        dcDF_dt = kTT * Xan * DR - kdel * DF

        return [dcXan_dt, dcEx_dt, dcDR_dt, dcDF_dt]
    
    def integrate(k1, times, init_values):
        result = odeint(solve, init_values, times, args=(k1,))
        Xan = result[:, 0]
        Ex = result[:, 1]
        DR = result[:, 2]
        DF = result[:, 3]
        return Xan, Ex, DR, DF
    
    def _max(k1, t):
        
        Xan, Ex, DR, DF = integrate(k1, t, [1, 0, 0, 0])
        DF_corr = DF * (Tall(A0 * Xan) if A0 > 0 else 1)
        
        if not curves:
            t_max_idx = np.argmax(DF_corr)
            min_idx = t_max_idx - 1 if t_max_idx > 0 else 0
            t_part = np.linspace(t[min_idx], t[t_max_idx + 1], times.shape[0])
            Xan, Ex, DR, DF = integrate(k1, t_part, [Xan[min_idx], Ex[min_idx], DR[min_idx], DF[min_idx]])
        
            DF_corr = DF * (Tall(A0 * Xan) if A0 > 0 else 1)
        
        if curves:  # return the all curve, 
            return DF_corr / np.max(DF_corr) if norm_curves else DF_corr
        
        return t_part[np.argmax(DF_corr)] if not intensity else np.max(DF_corr)
    
    t_num = t_points if curves else 100
#     t_max = 4e-6
    
    times = np.linspace(0, t_max, t_num)
    # define output array, either array (only maxima) or matrix (full curves)
    maxima = np.zeros(conc.shape[0] if not curves else (conc.shape[0], times.shape[0]))
        
    for i, k1 in enumerate(kq*conc):
        maxima[i] = _max(k1, times)
        
    return maxima

In [8]:
# k0 = 4.1e5
# kd = 1.86e5
# kq = 1.31e10

k0, kq, kd = get_constants(T=-30)

conc = np.linspace(1e-10, 7e-4, 100)
ll = lower_lim(conc, k0, kq, kd)

# x = np.linspace(0, 4e-6, 5000)
ul = upper_lim(conc, k0, kq, kd)

# (conc, kex, k_ex=0, kdel=1e6, kTT=1e10, k0=4.0e5, kd=1.86e5, kq=1.34e10, A0=0,
#             t_max=4e-6, intensity=False, curves=False, norm_curves=False, t_points=1000):

var_ = general(conc, 1.5e6, k_ex=0, kTT=1e10, k0=k0, kd=kd, kq=kq, kdel=1e6, A0=0, t_max=6e-6)
# var2 = general_2(conc, 2.5e6, 1e10, A0=0)
plt.plot(conc, ul, label='upper', lw=3)
plt.plot(conc, ll, label='lower', lw=3)
plt.plot(conc, var_, label='true-var', lw=3, ls=':', color='red')

# for d in np.linspace(3e5, 5e6, 10):
#     plt.plot(conc, general_2(conc, d, 1e11, A0=0), label=f'kdelay={d}', color='k')



# plt.plot(conc, ul - ll, label='diff')
plt.legend()
plt.xlabel('x label')
plt.ylabel('y label')


# plt.savefig('figure.png', dpi=1000, format='png', transparent=True)
plt.show()

In [14]:
_c = np.linspace(1e-8, 6e-4, 1000)
# _c = np.logspace(-8, -3, 1000)

k0, kq, kd = get_constants(T=-30)

t_points = 1000
t_max = 6e-6

matrix = general(_c, 1.5e+010, k_ex=0, kTT=1e10, k0=k0, kd=kd, kq=kq, kdel=1e6, A0=0.25,
                 t_max=t_max, curves=True, norm_curves=False, t_points=t_points)
# matrix = general_2(_c, 2.5e6, kTT=1e12, A0=0, t_max=6e-6, curves=True, norm_curves=False)

times = np.linspace(0, t_max, t_points)

matrix /= np.max(matrix)  # normalize to global maximum

curve_x = _c
curve_y = np.zeros(_c.shape[0]) # max time
curve_z = np.zeros(_c.shape[0]) # max intensity

# maxima curve
for i in range(_c.shape[0]):
    curve_y[i] = times[np.argmax(matrix[i])]
    curve_z[i] = np.max(matrix[i])

# matrix /= np.max(matrix)  # normalize to global maximum


x, y = np.meshgrid(_c, times)  # needed for pcolormesh to correctly scale the image

from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.ticker as mtick

%matplotlib qt

# X, Y, Z = axes3d.get_test_data(0.2)

# Normalize to [0,1]
norm = plt.Normalize(matrix.min(), matrix.max())
colors = cm.inferno_r(norm(matrix.T))
rcount, ccount, _ = colors.shape

fig = plt.figure()
ax = fig.gca(projection='3d')
# surf = ax.plot_surface(x, y, matrix.T, rcount=1, ccount=50,
#                        facecolors=colors, shade=False, linewidth=0.5)
# surf.set_facecolor((0,0,0,0))
wf = ax.plot_wireframe(x, y, matrix.T, rstride=0, cstride=27, linewidth=0.9, color='k')

curve_plot = ax.plot3D(curve_x, curve_y, curve_z, zdir='z', color='green', lw=5, zorder=10)

levels = np.linspace(-0.01, 1, 6)
cm = ax.contourf(x, y, matrix.T, zdir='z', offset=0, levels=levels, cmap='inferno_r')


# cc = ax.contourf(x, y, matrix.T, zdir='y', offset=-0.1)

# ax.colorbar(label='Relative intensity')

# plot data matrix D

# plt.pcolormesh(x, y, matrix.T, cmap='inferno_r')
# plt.colorbar(label='Relative intensity')
# plt.title("Data matrix $D$")
# plt.ylabel('Time [s]')
# plt.xlabel('CP1 concentration [M]')
# ax.zlabel('Normalized intensity')
ax.set_xlabel('CP1 concentration (M)', fontsize=16, rotation=0)
ax.set_ylabel('Time (s)', fontsize=15)
ax.set_zlabel('Norm. intensity', fontsize=15, rotation=0)
ax.set_zlim(0, 1)

# ax.ticklabel_format(axis='both', style='sci')
ax.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))


# plt.xscale('symlog', subsx=[1, 2, 3, 4, 5, 6, 7, 8, 9], linscalex=0.3, linthreshx=1e-5)
# plt.xscale('log')
# yaxis = plt.gca().yaxis
# yaxis.set_minor_locator(MinorSymLogLocator(100))

# plt.savefig('fig.pdf', format='pdf')
# plt.savefig('figure.png', dpi=1000, format='png', transparent=True)

plt.show()

